ENTRAINEMENT DU MODELE DE FORET ALEATOIRE (RANDOM FOREST)

Importation des librairies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import numpy as nu
from sklearn.model_selection import train_test_split # Pour le split train/test
from sklearn.ensemble import RandomForestClassifier # Modèle sélectionné (catégorisation/regression)

train = pd.read_csv('data/train.csv')

Nettoyage des données

In [17]:
# Extraction titre
train['Title'] = train['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)
train['Title'] = train['Title'].str.strip()
# Verification des entrées concernées
train[train['Age'].isnull()].groupby(['Title', 'Pclass', 'Sex', 'Parch']).size()
# Conversion données NULL 
age_medians = train.groupby(['Title', 'Pclass'])['Age'].median()
# Fonction pour imputer/remplacer les âges manquants en fonction des catégories concernées
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_medians.get((row['Title'], row['Pclass']), train['Age'].median())
    return row['Age']
# Application des changements à la table
train['Age'] = train.apply(impute_age, axis=1)
# Conversion du type de données de la colonne 'Sex' en valeurs numériques
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'])

# Conversion du type de données de la colonne 'Title' & création d'une table de correspondance pour comprendre les modifications
train['Title'] = le.fit_transform(train['Title'])
title_mapping = pd.DataFrame({
    'Title': le.classes_,
    'Encoded': le.transform(le.classes_)
})
title_mapping.sort_values('Encoded')

,Title,Encoded
0,Capt,0
1,Col,1
2,Don,2
3,Dr,3
4,Jonkheer,4
5,Lady,5
6,Major,6
7,Master,7
8,Miss,8
9,Mlle,9


MODELE 2 - Random Forest 

Sélection des features (X) et de la target (y)

In [18]:
features = ['Age', 'Sex', 'Pclass', 'Title']
X = train[features]

y = train['Survived']